In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [43]:
results="output_data/cities.csv"
results_df=pd.read_csv(results)
results_df['humidity'] = results_df['humidity'].astype(float)
print (results_df.dtypes)

city           object
lat           float64
lng           float64
max_temp      float64
humidity      float64
windspeed     float64
cloudiness      int64
country        object
date            int64
dtype: object


In [44]:
locations=results_df[["lat", "lng"]]
humidity=[]
for index, row in results_df.iterrows():
    humidity.append(row["humidity"])

In [45]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [46]:
# ideal conditions: max temp over 60, cloudiness less than 50%, humidity at less than 10

ideal_df=results_df.loc[(results_df["max_temp"]>60) & 
                        (results_df["cloudiness"]<50) & 
                        (results_df["humidity"]<10)]
ideal_df

,city,lat,lng,max_temp,humidity,windspeed,cloudiness,country,date
110,tessalit,20.20,1.01,111.88,6.0,10.07,0,ML,1592408109
238,arlit,18.74,7.39,106.48,7.0,18.12,0,NE,1592408176
259,riyadh,24.69,46.72,107.60,8.0,13.87,10,SA,1592408019
349,taoudenni,22.68,-3.98,112.96,6.0,18.19,0,ML,1592408192
460,iranshahr,27.20,60.68,113.00,6.0,9.17,20,IR,1592408212
468,sibi,29.54,67.88,108.09,7.0,7.16,0,PK,1592408213
495,pindi bhattian,31.90,73.27,102.34,8.0,5.35,0,PK,1592408217


In [60]:
hotel_df=ideal_df[["city","country", "lat", "lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,country,lat,lng,Hotel Name
110,tessalit,ML,20.20,1.01,
238,arlit,NE,18.74,7.39,
259,riyadh,SA,24.69,46.72,
349,taoudenni,ML,22.68,-3.98,
460,iranshahr,IR,27.20,60.68,
468,sibi,PK,29.54,67.88,
495,pindi bhattian,PK,31.90,73.27,


In [61]:

params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]

  
    params["location"] = f"{lat},{lng}"
 

    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

   
    name_address = requests.get(base_url, params=params)
    

  
    name_address = name_address.json()

    hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
hotel_df    

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,city,country,lat,lng,Hotel Name
110,tessalit,ML,20.20,1.01,Tessalit
238,arlit,NE,18.74,7.39,Arlit
259,riyadh,SA,24.69,46.72,Riyadh
349,taoudenni,ML,22.68,-3.98,Taoudenni
460,iranshahr,IR,27.20,60.68,Iranshahr
468,sibi,PK,29.54,67.88,Sibi
495,pindi bhattian,PK,31.90,73.27,Faisalabad
